In [1]:
import onnx
import numpy as np
import tvm
from tvm import relay
from tvm.contrib import graph_executor

In [2]:
onnx_model_path = "/home/pravinpb/pycode/MCW/Assignments/submissions/FTTU-interns/pravin/tvm_assignments/mnist_resnet_final.onnx"
onnx_model = onnx.load(onnx_model_path)

In [3]:
# Define target hardware
target = "llvm" 

In [6]:
input_name = "input"  # Replace with the actual input name in your ONNX model
input_shape = (1, 28,28, 3)  # Update to match your model's input shape

In [7]:
shape_dict = {input_name: input_shape}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

In [8]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [11]:
lib.export_library("optimized_model.so")

In [12]:
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

In [13]:
input_data = np.random.randn(*input_shape).astype("float32")
module.set_input(input_name, input_data)

In [14]:
# Perform inference
module.run()

In [15]:
output = module.get_output(0).asnumpy()
print("Inference output shape:", output.shape)

Inference output shape: (1, 10)


In [16]:
output

array([[1.6841732e-08, 4.1241266e-02, 7.8796811e-04, 7.6023571e-05,
        4.0135183e-07, 9.5712715e-01, 7.6569541e-04, 7.4818268e-10,
        1.4080050e-06, 8.0464730e-08]], dtype=float32)

In [18]:
with open("model_params.params", "wb") as f:
    f.write(relay.save_param_dict(params))

In [20]:
with open("optimized_model_structure.json","w") as f:
    f.write(lib.get_graph_json())